In [5]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from collections import OrderedDict
import pickle
import pickle5
import seaborn as sns

In [3]:
# get data
df = pd.read_pickle("../data/Electric_Vehicle_Charging_Station_Energy_Consumption_cleaned_covid.pkl")
df.head()

,Station_Name,Station_Address,City,State_Province,Postal_Code,Transaction_Date,Transaction_Start_Time,Charging_Time__minutes_,Energy__kWh_,GHG_Savings__kg_,...,Transaction_Date_Datetime,start_Datetime,end_Datetime,year,idle_time_minutes,latitude,longitude,weekday,weekday_name,distance_center_km
0,COMM VITALITY / 1104 SPRUCE1,1100 Spruce St,Boulder,Colorado,80302,2018/01/31 00:00:00+00,2021/01/21 17:43:00+00,128.0,2.84,1.19,...,2018-01-31 00:00:00+00:00,2018-01-31 17:43:00,2018-01-31 19:51:00,2018,929.0,40.018482,-105.281066,2,Wednesday,0.152203
1,COMM VITALITY / 1000WALNUT1,900 Walnut St,Boulder,Colorado,80302,2018/01/31 00:00:00+00,2021/01/21 17:16:00+00,68.0,5.66,2.38,...,2018-01-31 00:00:00+00:00,2018-01-31 17:16:00,2018-01-31 18:24:00,2018,228.0,40.015994,-105.282473,2,Wednesday,0.378654
2,BOULDER / REC CENTER ST2,1305-1335 Gillaspie Dr,Boulder,Colorado,80305,2018/01/31 00:00:00+00,2021/01/21 13:52:00+00,73.0,3.94,1.65,...,2018-01-31 00:00:00+00:00,2018-01-31 13:52:00,2018-01-31 15:05:00,2018,2356.0,39.974881,-105.249219,2,Wednesday,5.638648
3,BOULDER / BASELINE ST1,900 Baseline Rd,Boulder,Colorado,80302,2018/01/31 00:00:00+00,2021/01/21 13:54:00+00,40.0,4.42,1.86,...,2018-01-31 00:00:00+00:00,2018-01-31 13:54:00,2018-01-31 14:34:00,2018,1259.0,40.000148,-105.282437,2,Wednesday,2.096847
4,BOULDER / BASELINE ST1,900 Baseline Rd,Boulder,Colorado,80302,2018/01/31 00:00:00+00,2021/01/21 11:51:00+00,42.0,4.60,1.93,...,2018-01-31 00:00:00+00:00,2018-01-31 11:51:00,2018-01-31 12:33:00,2018,81.0,40.000148,-105.282437,2,Wednesday,2.096847


In [6]:
with open("../data/idle_times.pickle", "rb") as handle:
    station_dict = pickle5.load(handle)

In [18]:
### Usefull function:
def dayNameFromWeekday(weekday):
    if weekday == 0:
        return "Monday"
    if weekday == 1:
        return "Tuesday"
    if weekday == 2:
        return "Wednesday"
    if weekday == 3:
        return "Thursday"
    if weekday == 4:
        return "Friday"
    if weekday == 5:
        return "Saturday"
    if weekday == 6:
        return "Sunday"

In [19]:
names = df['Station_Name'].unique()

In [45]:
## convert the station names
temp = []

for i in range(0,len(df)):
    name = df.iloc[i]['Station_Name'] 
    if name == 'BOULDER / BOULDER PARK S1' or name == 'BOULDER / BOULDER PARK S2':
        temp.append('BOULDER / BOULDER PARK')
    elif name == 'BOULDER / REC CENTER ST1' or name == 'BOULDER / REC CENTER ST2':
        temp.append('BOULDER / REC CENTER')
    elif name == 'COMM VITALITY / 1000WALNUT1' or name == 'COMM VITALITY / 1000WALNUT2':
        temp.append('COMM VITALITY / 1000WALNUT')
    elif name == 'COMM VITALITY / 1500PEARL1' or name == 'COMM VITALITY / 1500PEARL2':
        temp.append('COMM VITALITY / 1500PEARL')
    else:
        temp.append(name)
        
df['new_Station_Name'] = temp

In [46]:
## Usefull function:
def charging_ports(X):
    if X == 'BOULDER / BOULDER PARK':
        return 4
    elif X == 'BOULDER / REC CENTER ST':
        return 4
    elif X == 'COMM VITALITY / 1000WALNUT':
        return 4
    elif X == 'COMM VITALITY / 1500PEARL':
        return 4
    else:
        return 2

### Idle-times

In [47]:
names = df['new_Station_Name'].unique()

for j in range (0,len(names)):
    bob = station_dict[names[j]]
    birth = []
    death = []
    # As we have the activity for every minute we iterate through it.
    # When we observe a change from 0 to other we save the birth and death time. 
    for i in range (1, len(bob['t0'])):
        if bob['nobs'][i] == 0 and bob['nobs'][i-1] > 0:
            birth.append(bob['t0'][i])
        elif bob['nobs'][i-1] == 0 and bob['nobs'][i] > 0:
            death.append(bob['t1'][i-1])

    # check if number of death and births are the same
    if len(birth) != len(death):
        print("Problem!!", names[j])
        #break

    # calculate life/idle times
    idle_time = []
    for i in range(0,len(birth)):
        time_diff = death[i] - birth[i]
        time_diff_min = time_diff.days*24*60 + time_diff.seconds/60
        idle_time.append(time_diff_min)

    # construct dataframe
    df_temp = pd.DataFrame({'life_time':idle_time,'datetime':birth})
    df_temp['latitude'] = [df[df['new_Station_Name'] == names[j]].latitude.unique()[0]]*len(birth) # add latitude
    df_temp['longitude'] = [df[df['new_Station_Name'] == names[j]].longitude.unique()[0]]*len(birth) # add longitude
    df_temp['distance_center_km'] = [df[df['new_Station_Name'] == names[j]].distance_center_km.unique()[0]]*len(birth) # add distance to center (km)
    df_temp['Station_Name'] = [names[j]]*len(birth) # add station name
    df_temp['weekday_b'] = df_temp['datetime'].dt.dayofweek
    df_temp['weekday_b_name'] = df_temp['weekday_b'].apply(dayNameFromWeekday)
    df_temp['hour_b'] = df_temp['datetime'].dt.hour
    df_temp['charging_ports'] = df_temp['Station_Name'].apply(charging_ports)
    
    # concat dataframes
    if j == 0: # if it is the first station define new data frame
        df_uu = df_temp
    else: # otherwise add the temporary dataframes
        df_uu = pd.concat([df_uu,df_temp], names=column_names)

# save dataframe
df_uu.to_pickle("../data/under_utilization_combinedStations.pkl")
        
df_uu.shape

(12711, 10)